In [8]:
import pandas as pd
import numpy as np
import os
import re

pd.set_option('display.max_rows', 50)

In [9]:
# CALCULATE BUDGETS FOR EACH AREA

month = '2022-05'

init_budget = {}

init_budget['governance'] = 2
init_budget['curation'] = 2.5
init_budget['marketing & pr'] = 2.5
init_budget['community'] = 1.5
init_budget['finance'] = 1.5

perf_tranche = {}

perf_tranche['governance'] =  0.4
perf_tranche['curation'] = 0.4
perf_tranche['marketing & pr'] = 0.4
perf_tranche['community'] = 0.5
perf_tranche['finance'] = 0.4

perf_score = {}

perf_score['governance'] =  1/3
perf_score['curation'] = 3/3
perf_score['marketing & pr'] = 6/8
perf_score['community'] = 1/2
perf_score['finance'] = 2/3


final_budget = {}
total_final_budget = 0
total_init = 0

for area in init_budget:
  final_budget[area] = round(init_budget[area]*(1-perf_tranche[area]) + init_budget[area]*(perf_tranche[area]*perf_score[area]),2)
  total_final_budget = total_final_budget + final_budget[area]
  total_init = total_init + init_budget[area]

print(final_budget)
print("total final budget = " + str(round(total_final_budget,2)) + " out of " + str(total_init))

{'governance': 1.47, 'curation': 2.5, 'marketing & pr': 2.25, 'community': 1.12, 'finance': 1.3}
total final budget = 8.64 out of 10.0


In [10]:
# READ AND CLEAN COORDINAPE DATA
dfs = {}
voting_members = {}
total_give = {}

for file in os.listdir('./coordinape_data'):
  try:
    # read data from coordinape files  
    key = re.search('DAO-(.+?)-', file).group(1).lower()
    # print(key)
    dfs[key] = pd.read_csv('./coordinape_data/'+ file)
    
    # calculate voting stats for each area
    voting_members[key] = int(dfs[key]['sent'].astype(bool).sum(axis=0))
    total_give[key] = int(dfs[key]['received'].sum())

    # calculate compensation per area
    dfs[key] = dfs[key].sort_values(by=['received'], ascending = False) # sort
    dfs[key] = dfs[key][['name','address','received']] # select relevant cols
    dfs[key]['percent_comp'] = dfs[key]['received']/total_give[key] # calculate percentage comp.
    dfs[key]['eth_compensation'] = dfs[key]['percent_comp'] * final_budget[key] # calculate actual ETH comp.
    dfs[key] = dfs[key][dfs[key]['eth_compensation'] > 0] # filter people with 0 compensation
    dfs[key]['area'] = key
    dfs[key]['total_budget_of_area'] = final_budget[key]
  except NameError:
    print(file + ' -> file skipped')
    print(key)
    print(NameError)

# prepare area breakdown for reporting
df_export = pd.concat(dfs)

# merge everything and calculate compensation per member
comp_df = pd.concat(dfs)
comp_df = comp_df[['name','address','eth_compensation']]
comp_df = comp_df.groupby(by=['name','address']).sum().reset_index()
comp_df = comp_df.sort_values(by=['eth_compensation'], ascending = False)

print('# of members who voted per area: ' + str(voting_members))
print('total votes per area: ' + str(total_give))
print("total compensated members: " + str(len(comp_df)))

# of members who voted per area: {'community': 10, 'curation': 17, 'finance': 10, 'governance': 11, 'marketing & pr': 12}
total votes per area: {'community': 1260, 'curation': 1954, 'finance': 1235, 'governance': 1320, 'marketing & pr': 1532}
total compensated members: 31


In [11]:
# PRINT VOTES PER MEMBER PER AREA
df_export[['name','total_budget_of_area','received','percent_comp','eth_compensation']]


name  total_budget_of_area  received  \
community      47  matthewbrooks.eth                  1.12       201   
               46       brileigh.eth                  1.12       191   
               30        JoseCactuss                  1.12       180   
               17       claudia#0273                  1.12       130   
               29      samcronenberg                  1.12       107   
...                              ...                   ...       ...   
marketing & pr 17      MesutNFT#9591                  2.25        15   
               37          b123#2778                  2.25        15   
               29             vcolak                  2.25        10   
               44         glory#5406                  2.25         2   
               25    tommydavidphoto                  2.25         1   

                   percent_comp  eth_compensation  
community      47      0.159524          0.178667  
               46      0.151587          0.169778  
               30      0.142857          0.160000  
               17      0.103175          0.115556  
               29      0.084921          0.095111  
...                         ...               ...  
marketing & pr 17      0.009791          0.022030  
               37      0.009791          0.022030  
               29      0.006527          0.014687  
               44      0.001305          0.002937  
               25      0.000653          0.001469  

[98 rows x 5 columns]

In [12]:
# PRINT ETH COMPENSATION PER MEMBER FOR THIS MONTH
comp_df[['name','eth_compensation']]

,name,eth_compensation
9,brileigh.eth,1.510145
21,matthewbrooks.eth,1.422313
2,JoseCactuss,0.912924
13,claudia#0273,0.835399
23,munzmeister,0.689335
22,mishaderidder,0.382763
3,Lapsus#4632,0.368729
10,bthemouth#0088,0.340495
20,lucaspon,0.263932
15,darkroom.felix,0.253223


In [13]:
# CALCULATE RANKINGS FOR LONG TERM COMP MODEL

df_rankings = df_export.reset_index()

# This is where we get into a discussion of how exactly to normalize the ranking. The approach I've chosen here is to weigh each member's score by the areas budget, 
# which is equivalent to using their ETH compensation as the score measurement. This way, members who are performing in areas with higher budget (deemed more important) 
# will score higher in the rankings.

df_rankings['normalized_score'] = df_rankings['percent_comp'] * df_rankings['total_budget_of_area']/total_final_budget # score weighted by area's budget
df_rankings = df_rankings[['name','address','normalized_score','area']]
df_rankings = df_rankings.groupby(by=['name','address']).sum()
df_rankings = df_rankings.sort_values(by=['normalized_score'], ascending = False).reset_index()
df_rankings.index = df_rankings.index + 1
df_rankings['percentile'] = df_rankings.index/len(df_rankings)

# Award Conditions
conditions = [
    (df_rankings['percentile'] <= 0.1),
    (df_rankings['percentile'] > 0.1) & (df_rankings['percentile'] <= 0.25),
    (df_rankings['percentile'] > 0.25) & (df_rankings['percentile'] <= 0.5),
    (df_rankings['percentile'] > 0.5) & (df_rankings['percentile'] <= 0.8),
    (df_rankings['percentile'] > 0.8)
]

tier = [ 
    'top 10%',
    'top 25%',
    'top 50%',
    'bottom 50%',
    'bottom 20%'
]

tokens_awarded = [
    0,
    0,
    0,
    0,
    0
]

coordinape = [ 
    20,
    10,
    5,
    0,
    'reset to 100'
]


df_rankings['tier'] = np.select(conditions, tier)
df_rankings['$RAW Awarded'] = np.select(conditions, tokens_awarded)
df_rankings['coordinape_change'] = np.select(conditions, coordinape)


print('total $RAW Awarded = ' + str(df_rankings['$RAW Awarded'].sum()))
df_rankings[['name','normalized_score', 'percentile', 'tier', '$RAW Awarded', 'coordinape_change']]


total $RAW Awarded = 0


,name,normalized_score,percentile,tier,$RAW Awarded,coordinape_change
1,brileigh.eth,0.174785,0.032258,top 10%,0,20
2,matthewbrooks.eth,0.164620,0.064516,top 10%,0,20
3,JoseCactuss,0.105662,0.096774,top 10%,0,20
4,claudia#0273,0.096690,0.129032,top 25%,0,10
5,munzmeister,0.079784,0.161290,top 25%,0,10
6,mishaderidder,0.044301,0.193548,top 25%,0,10
7,Lapsus#4632,0.042677,0.225806,top 25%,0,10
8,bthemouth#0088,0.039409,0.258065,top 50%,0,5
9,lucaspon,0.030548,0.290323,top 50%,0,5
10,darkroom.felix,0.029308,0.322581,top 50%,0,5


In [16]:
# EXPORT RESULTS TO EXCEL
from pathlib import Path
Path("./outputs").mkdir(parents=True, exist_ok=True)

df_export.to_excel("./outputs/"+ month +" - compensation_per_area.xlsx")
comp_df.to_excel("./outputs/"+ month +" - compensation_by_member.xlsx")
df_rankings.to_excel("./outputs/"+ month +" - LT_comp_rankings.xlsx")